
## Deep Dream 

In [1]:
import tensorflow as tf
import numpy as np

c:\users\b.mehdi\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\b.mehdi\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\b.mehdi\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
from keras.applications import inception_v3
from keras import backend as K
import keras


In [ ]:
model = inception_v3.InceptionV3(weights='imagenet',include_top=False)
model.training=False

### Paramétrage de la configuration de Dream 

In [19]:
layer_settings = {
    "mixed4": 1.0,
    "mixed5": 1.5,
    "mixed6": 2.0,
    "mixed7": 2.5,
}
outputs_dict = dict(
    [
        (layer.name, layer.output)
        for layer in [model.get_layer(name) for name in layer_settings.keys()]
    ]
)
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

### Définition de la perte à maximiser 

In [20]:
def compute_loss(input_image):
    features = feature_extractor(input_image)
    loss = tf.zeros(shape=())
    for name in features.keys():
        coeff = layer_settings[name]
        activation = features[name]
        loss += coeff * tf.reduce_mean(tf.square(activation[:, 2:-2, 2:-2, :]))
    return loss

### Le processus d'ascension du gradient 

In [23]:
@tf.function
def gradient_ascent_step(image, learning_rate):
    with tf.GradientTape() as tape:
        tape.watch(image)
        loss = compute_loss(image)
    grads = tape.gradient(loss, image)
    grads = tf.math.l2_normalize(grads)
    image += learning_rate * grads
    return loss, image


def gradient_ascent_loop(image, iterations, learning_rate, max_loss=None):
    for i in range(iterations):
        loss, image = gradient_ascent_step(image, learning_rate)
        if max_loss is not None and loss > max_loss:
            break
        print(f"...Valeur de perte à {i}: {loss:.2f}")
    return image

In [30]:
step = 20.
num_octave = 3
octave_scale = 1.4
iterations = 30
max_loss = 15.
base_image_path = 'h2m.jpg'

### Ascension de gradients sur différentes échelles successives 

In [25]:
def preprocess_image(image_path):
    img = keras.utils.load_img(image_path)
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.inception_v3.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((img.shape[1], img.shape[2], 3))
    img /= 2.0
    img += 0.5
    img *= 255.
    img = np.clip(img, 0, 255).astype("uint8")
    return img

In [31]:
original_img = preprocess_image(base_image_path)
original_shape = original_img.shape[1:3]

successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]

shrunk_original_img = tf.image.resize(original_img, successive_shapes[0])

img = tf.identity(original_img)
for i, shape in enumerate(successive_shapes):
    print(f"Traitement de l'image à l'octave {i} de forme: {shape}")
    img = tf.image.resize(img, shape)
    img = gradient_ascent_loop(
        img, iterations=iterations, learning_rate=step, max_loss=max_loss
    )
    upscaled_shrunk_original_img = tf.image.resize(shrunk_original_img, shape)
    same_size_original = tf.image.resize(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = tf.image.resize(original_img, shape)
    keras.utils.save_img("h2m_deepdream/dream_at_scale_" + str(shape) + ".png", deprocess_image(img.numpy()))
keras.utils.save_img("h2m_deepdream/dream_final.png", deprocess_image(img.numpy()))

Traitement de l'image à l'octave 0 de forme: (183, 316)
...Valeur de perte à 0: 1.23
...Valeur de perte à 1: 1.35
...Valeur de perte à 2: 1.62
...Valeur de perte à 3: 1.98
...Valeur de perte à 4: 2.34
...Valeur de perte à 5: 2.80
...Valeur de perte à 6: 3.34
...Valeur de perte à 7: 3.87
...Valeur de perte à 8: 4.60
...Valeur de perte à 9: 5.37
...Valeur de perte à 10: 6.13
...Valeur de perte à 11: 7.16
...Valeur de perte à 12: 8.01
...Valeur de perte à 13: 9.02
...Valeur de perte à 14: 10.04
...Valeur de perte à 15: 10.83
...Valeur de perte à 16: 11.86
...Valeur de perte à 17: 12.97
...Valeur de perte à 18: 14.19
Traitement de l'image à l'octave 1 de forme: (257, 442)
...Valeur de perte à 0: 1.78
...Valeur de perte à 1: 3.27
...Valeur de perte à 2: 4.78
...Valeur de perte à 3: 6.56
...Valeur de perte à 4: 8.64
...Valeur de perte à 5: 12.58
Traitement de l'image à l'octave 2 de forme: (360, 620)
...Valeur de perte à 0: 4.66
...Valeur de perte à 1: 10.42



## Style transfer 

In [4]:
from keras.utils import load_img, img_to_array

base_image_path = 'h2m.jpg'

style_reference_image_path = 'starry_night.jpg'

original_width, original_height = load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height)



In [14]:
from keras.applications import vgg19

def preprocess_image(image_path):
    img = keras.utils.load_img(
        image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((img_height, img_width, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    return img

### Utilisation du réseau VGG19 pré-entraîné

In [15]:
   
model = vgg19.VGG19(weights='imagenet',include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

### La perte de contenu

In [16]:
def content_loss(target, combination):
    return K.sum(K.square(combination - target))

### La perte de style

In [17]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

### La perte de variation totale

In [18]:
def total_variation_loss(x):
    a = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

### Définition de la perte finale que nous allons minimiser

In [29]:

content_layer = 'block5_conv2'

style_layers = ['block1_conv1','block2_conv1',
                'block3_conv1','block4_conv1',
                'block5_conv1']

total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

@tf.function
def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    for layer_name in style_layers:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layers)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss



### Le processus de descente du gradient

In [ ]:
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

optimizer = tf.keras.optimizers.SGD(tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96))

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))

iterations = 4000
for i in range(1, iterations + 1):
    print(i)
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 10 == 0:
        print(f"Iteration {i}: loss={loss:.2f}")
        img = deprocess_image(np.array(combination_image))
        fname = f"h2m_style/combination_image_at_iteration_{i}.png"
        keras.utils.save_img(fname, img)